# LangGraph Message Accumulation with Reducers

This notebook demonstrates how **LangGraph reducers** enable
persistent conversational state by **accumulating messages**
instead of overwriting them.

The focus is on:
- Using `add_messages` as a reducer
- Understanding `Annotated` state fields
- Building a multi-turn conversational graph
- Conditional routing with preserved context


In [ ]:
import getpass
import os
from langgraph.graph import START, END, StateGraph, add_messages
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from collections.abc import Sequence
from typing import Literal, Annotated

In [ ]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

## Message Reducer: `add_messages`

`add_messages` is a **reducer function** that defines how
two lists of messages are merged.

It takes:
1. The existing list of messages
2. The newly returned messages

and returns a **combined list**.


In [ ]:
my_list = add_messages([HumanMessage("Hi! I'm Oscar. "), 
                        AIMessage("Hey, Oscar! How can I assist you?")], 
                        [HumanMessage("Could you summarize today's news?")])

In [ ]:
my_list

## Defining a Reducer-Aware State

By default, LangGraph **replaces** state values after each node.

Here, we want **new messages to be appended** instead.

Using `Annotated`, we attach the `add_messages` reducer
to the `messages` field so that updates are merged
instead of overwritten.


In [ ]:
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

## Chat Model Initialization

A deterministic chat model is used to generate responses
within graph nodes.


In [ ]:
chat = ChatOpenAI(
    model="gpt-5-nano", 
    temperature=0, 
    model_kwargs= {"text":{"verbosity": 'low'}},
    ) 


## Defining Graph Nodes

Each node:
- Reads the current state
- Performs an action
- Returns new messages to be merged into state


In [ ]:
def ask_question(state: State) -> State:
    
    print(f"\n-------> ENTERING ask_question:")
    for i in state["messages"]:
        i.pretty_print()
    question = "What is your question?"

    print(question)
    
    return State(messages = [AIMessage(question), HumanMessage(input())])

In [ ]:
def chatbot(state: State) -> State:
    
    print(f"\n-------> ENTERING chatbot:")
    for i in state["messages"]:
        i.pretty_print()
    
    response = chat.invoke(state["messages"])
    response.pretty_print()
    
    return State(messages = [response])

In [ ]:
def ask_another_question(state: State) -> State:
    
    print(f"\n-------> ENTERING ask_another_question:")
    for i in state["messages"]:
        i.pretty_print()
    
    question = "Would you like to ask one more question (yes/no)?"
    print(question)
    
    return State(messages = [AIMessage(question), HumanMessage(input())])

## Conditional Routing Function

The routing function determines whether the graph:
- Continues the conversation
- Terminates execution


In [ ]:
def routing_function(state: State) -> Literal["ask_question", "__end__"]:
    
    if state["messages"][-1].content == "yes":
        return "ask_question"
    else:
        return "__end__"

## Building the Conversational Graph

Nodes are connected using directed edges.
Conditional edges control looping behavior.


In [ ]:
graph = StateGraph(State)

In [ ]:
graph.add_node("ask_question", ask_question)
graph.add_node("chatbot", chatbot)
graph.add_node("ask_another_question", ask_another_question)

graph.add_edge(START, "ask_question")
graph.add_edge("ask_question", "chatbot")
graph.add_edge("chatbot", "ask_another_question")
graph.add_conditional_edges(source = "ask_another_question", 
                            path = routing_function)

In [ ]:
graph_compiled = graph.compile()

In [ ]:
graph_compiled

## Executing the Graph

The graph is invoked with an empty initial state.
Messages accumulate across turns due to the reducer.


In [ ]:
graph_compiled.invoke(State(messages = []))

## Summary

This notebook demonstrated:

- Using `add_messages` as a reducer
- Attaching reducers via `Annotated`
- Preserving conversation history across nodes
- Building a looping conversational graph
- Conditional routing with accumulated state

Reducers enable LangGraph to act as a true
multi-turn conversational engine.
